<h1>One Hot Encoding und Dummy-Variablen</h1>

Wenn ein Dataset Text enthält, egal ob der Text eine numerische Bedeutung hat wie "Zwei", muss dieser in eine numerische Repräsentation umgeformt werden, damit die Modelle damit umgehen könne. 

Hier wird der Fall betrachtet, dass es nur einzelne Wörter sind.

<i>Abb1:</i> Übersicht der Datentypen 
<center>
    <img src="./files_data/img/DataTypes_encode_Notebook_1.PNG" width=800 hight=1000 >
</center>

Bei einem gegebenen Dataset mit verschiedenen Stadtnamen, wird OnHotEncode angewendet. 

<i>Abb2:</i> On Hot Encode


<img src="./files_data/img/DataTypes_encode_Notebook_2.PNG" width=600 hight=5000 >


<i>Abb3:</i> Integer / Label Encoding


<img src="./files_data/img/DataTypes_encode_Notebook_3.PNG" width=500 hight=400 >

On Hot Encoding: weise binäre Werte zu wie 0 und 1 oder True und False. <br>Für jede Kategorie eine Spalte.

Die erstellten Variablen nennt man auch Dummy-Variablen

Label Encoding: weise fortlaufende Zahlen zu, die je eine Stadt repräsentiert. <br>Eine Spalte für jede Kategorie

Die nominalen Daten folgen keiner bestimmten Reihenfolge und haben keine Relation zueinander. Daher wird hier eher On Hot Encoding genutzt.

Bei den anderen, wo es eine Reihenfolge gibt, wie zum Beispiel "Bachelor, Master, ...", kann ein Label Encoding angewendet werden.

Bei dem Label Encoding kann eine Relation zwischen den Daten hergestellt werden (Köln + Alsfeld = Fulda oder Köln > Fulda). Diese Annahmen können zu falschen Schlussfolgerungen führen.

Sklearn und Pandas bieten gute Möglichkeiten, die betroffenen Spalten umzuformen. 

Beispiel Dataset: https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction [Letzter Zugriff 04.06.2024] 

> fedesoriano. (September 2021). Heart Failure Prediction Dataset. Retrieved [Date Retrieved]<br> from https://www.kaggle.com/fedesoriano/heart-failure-prediction.

Dieses Dataset kann gut für die Veranschaulichung genutzt werden. 

In [35]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [14]:
heart_data = pd.read_csv("./files_data/data/heart_failure_prediction.zip", compression='zip')
heart_data.head(2)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1


In [15]:
heart_data2 = heart_data[ ['Sex', 'ChestPainType', 'ST_Slope' ] ] # 'ChestPainType', 'ST_Slope'
heart_data2.head(3)

,Sex,ChestPainType,ST_Slope
0,M,ATA,Up
1,F,NAP,Flat
2,M,ATA,Up


Mit nur einem Aufruf erstellt Pandas die Dummy-Variablen. Der Datentyp kann von Boolean zu Integer umgestellt werden.

In [16]:
# Gibt die Tabelle mit den Dummies zurück.
# - On Hot Encode - #
pd.get_dummies(heart_data['Sex']) # dtype="int" => Int statt Bool.

,F,M
0,False,True
1,True,False
2,False,True
3,True,False
4,False,True
...,...,...
913,False,True
914,False,True
915,False,True
916,True,False


Diese Tabelle kann vielseitig genutzt werden. Um es für das Training zu nutzen, wird es an das Dataframe angeknüpft, welches für das Training genutzt wird.

In [17]:
# Erstelle Dummies, dann füge die Dataframes zusammen. 
sex_column_dummy = pd.get_dummies(heart_data['Sex'], dtype="int")
heart_data2 = pd.concat([heart_data2, sex_column_dummy], axis="columns")
heart_data2.head()

,Sex,ChestPainType,ST_Slope,F,M
0,M,ATA,Up,0,1
1,F,NAP,Flat,1,0
2,M,ATA,Up,0,1
3,F,ASY,Flat,1,0
4,M,NAP,Up,0,1


<i>Abb4:</i> 2 Spalten müssen weg.

<img src="./files_data/img/DataTypes_encode_Notebook_4.PNG" width=300 hight=300 >

Der nächste Schritt ist, die Spalte "Sex" zu löschen, da diese nicht mehr benötigt wird. 

Eine der Spalten "F" oder "M" muss ebenfalls entfernt werden, da es sonst Redundanz und Multilinearität erzeugt => Dummy Variable Trap.

In [18]:
# 2 Spalten werden gelöscht
heart_data3 = heart_data2.drop(['Sex', 'F'], axis="columns")
heart_data3.head(3)

,ChestPainType,ST_Slope,M
0,ATA,Up,1
1,NAP,Flat,0
2,ATA,Up,1


Dasselbe ist auch mit Sklearn möglich. 

In [25]:
chestPainType = heart_data3['ChestPainType']  # Separat
chestPainType.unique()                        # 4 Werte 

array(['ATA', 'NAP', 'ASY', 'TA'], dtype=object)

In [29]:
import numpy as np

In [34]:
# Integer / Label Encoder # 
label_enc = LabelEncoder()  # Encoder 

encoded_chestPainTypelabel   = label_enc.fit_transform(chestPainType)  # Erstelle Encoding

np.unique(encoded_chestPainTypelabel) # Ergebnis-  0 bis 3 für diese 4 Werte. Kein On Hot Encode.

array([0, 1, 2, 3])

In [43]:
from sklearn.compose import ColumnTransformer

In [44]:
ct = ColumnTransformer([("ChestPainType", OneHotEncoder(), [0])], remainder = 'passthrough')

ct.fit_transform(heart_data3)

array([[0.0, 1.0, 0.0, 0.0, 'Up', 1],
       [0.0, 0.0, 1.0, 0.0, 'Flat', 0],
       [0.0, 1.0, 0.0, 0.0, 'Up', 1],
       ...,
       [1.0, 0.0, 0.0, 0.0, 'Flat', 1],
       [0.0, 1.0, 0.0, 0.0, 'Flat', 0],
       [0.0, 0.0, 1.0, 0.0, 'Up', 1]], dtype=object)

In [42]:
# On Hot Encoder # 
sklearn_ohe = OneHotEncoder(categories=[0]) 

sklearn_ohe.fit_transform(heart_data3)

ValueError: Shape mismatch: if categories is an array, it has to be of shape (n_features,).